In [7]:
from tqdm import tqdm
import time
from CGWODA import HybridOptimizationAlgorithm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import preprocessing
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.neighbors import KNeighborsClassifier
from zoofs import DragonFlyOptimization

from zoofs import GreyWolfOptimization
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from zoofs.baseoptimizationalgorithm import BaseOptimizationAlgorithm
from zoofs import DragonFlyOptimization

In [2]:
from sklearn.metrics import log_loss
def objective_function_topass( model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train) 
    y_pred = model.predict(X_test)
    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    # Calculate the classification error rate
    error_rate = 1 - accuracy
    cardinality_R = X_train.shape[1]
    # Calculate the total number of features in C
    total_features_C = 10
    alpha = 0.99
    # Calculate beta
    beta = 1 - alpha
    # Calculate the fitness value
    fitness = error_rate * alpha + beta * (cardinality_R / total_features_C)
    return fitness
    

In [ ]:
def run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test):
    results_per_iteration = []
    best_feature_lists = []
    best_dims = []
    best_scores = []

    for _ in tqdm(range(num_iterations), desc="Iterations", unit="iteration"):
        hybrid_object = HybridOptimizationAlgorithm(
            objective_function=objective_function_topass,
            n_iteration=100,
            population_size=11,
            logger=None,
        )
        
        knn_classifier = KNeighborsClassifier()
        
        hybrid_object.fit(knn_classifier, X_train, y_train, X_test, y_test, verbose=False)
        
        
        iteration_results = {
            "results_per_iteration": hybrid_object.best_results_per_iteration,
            "best_dim_list": hybrid_object.best_dim.tolist(),
            "best_feature_list": hybrid_object.best_feature_list,
            "best_score": hybrid_object.best_score
        }
        
        best_feature_list = hybrid_object.best_feature_list
        best_dim = hybrid_object.best_dim
        best_score = hybrid_object.best_score
        
        results_per_iteration.append(iteration_results)
        best_feature_lists.append(best_feature_list)
        best_dims.append(best_dim.tolist())
        best_scores.append(best_score)
    
    return results_per_iteration, best_feature_lists, best_dims, best_scores

## Dataset 1

In [3]:
def preprocess_dataset(dataset):
        # Separate the features and target variable
        if 'id' in dataset.columns:
            dataset = dataset.drop('id', axis=1)

        # Perform label encoding on categorical features
        categorical_features = dataset.select_dtypes(include=['object']).columns
        label_encoder = LabelEncoder()
        for feature in categorical_features:
            dataset[feature] = label_encoder.fit_transform(dataset[feature])

        dataset['bmi'].fillna(dataset['bmi'].mean(), inplace=True)
        
        # Normalize the data
        scaler = MinMaxScaler()
        normalized = scaler.fit_transform(dataset)

        # Return the preprocessed dataset
        preprocessed_dataset = pd.DataFrame(dataset, columns=dataset.columns)
        return preprocessed_dataset

In [4]:
stroke=pd.read_csv('stroke4.csv')
stroke=preprocess_dataset(stroke)
X = stroke.drop(['stroke'],axis=1)
y = stroke['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [9]:
num_iterations = 30 
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations: 100%|██████████| 30/30 [26:20<00:00, 52.68s/iteration]


In [ ]:
scores

In [10]:
features

[['gender', 'heart_disease', 'ever_married'],
 ['age', 'hypertension', 'ever_married'],
 ['age', 'heart_disease'],
 ['gender', 'hypertension', 'heart_disease'],
 ['gender', 'hypertension', 'heart_disease'],
 ['gender', 'ever_married'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['heart_disease', 'ever_married'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['gender', 'ever_married', 'work_type'],
 ['gender', 'hypertension', 'heart_disease', 'ever_married'],
 ['gender', 'hypertension', 'ever_married'],
 ['heart_disease', 'work_type', 'Residence_type'],
 ['gender', 'heart_disease'],
 ['age', 'ever_married'],
 ['gender', 'heart_disease', 'ever_married'],
 ['gender'],
 ['gender', 'heart_disease'],
 ['hypertension', 'heart_disease'],
 ['hypertension', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'ever_married'],
 ['gender', 'heart_disease'],
 ['gender', 'hypertension', 'ever_married'],
 ['gender', 'hypertension'],
 ['age', 'hyper

In [11]:
from collections import Counter

# Convert sublists to tuples for counting
tuple_list = [tuple(sublist) for sublist in features]
counter = Counter(tuple_list)
most_common =list( counter.most_common(1)[0][0])
print(most_common)  




['gender', 'heart_disease', 'ever_married']


In [12]:
Stroke_data =pd.DataFrame(X[['age', 'heart_disease', 'ever_married', 'avg_glucose_level', 'bmi']])
Stroke_data['stroke']=y

In [13]:
predictor_combined = TabularPredictor(
    label="stroke", 
    
).fit(
    train_data=Stroke_data, 

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_110907\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_110907\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   33.12 GB / 255.46 GB (13.0%)
Train Data Rows:    9722
Train Data Columns: 5
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1313.79 MB


[1000]	valid_set's binary_error: 0.101747
[2000]	valid_set's binary_error: 0.0883864
[3000]	valid_set's binary_error: 0.0801644
[4000]	valid_set's binary_error: 0.0719424
[5000]	valid_set's binary_error: 0.0688592
[6000]	valid_set's binary_error: 0.0678314
[7000]	valid_set's binary_error: 0.0668037
[8000]	valid_set's binary_error: 0.0647482
[9000]	valid_set's binary_error: 0.0626927
[10000]	valid_set's binary_error: 0.0626927


	0.9394	 = Validation score   (accuracy)
	12.99s	 = Training   runtime
	0.93s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.045221


	0.9609	 = Validation score   (accuracy)
	1.58s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9301	 = Validation score   (accuracy)
	1.62s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9311	 = Validation score   (accuracy)
	2.15s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.9661	 = Validation score   (accuracy)
	7.51s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9342	 = Validation score   (accuracy)
	1.08s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9301	 = Validation score   (accuracy)
	1.16s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8571	 = Validation score   (accuracy)
	20.2s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.9363	 = Validation score   (accuracy)
	1.42s	 = Training   runtime
	0.02s	 =

In [23]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)


Iterations:   3%|▎         | 1/30 [01:12<34:52, 72.14s/iteration] [ 2023-08-21 11:27:32,841 ] 1 individuals went zero
 [ 2023-08-21 11:27:41,057 ] 1 individuals went zero
 [ 2023-08-21 11:27:41,281 ] 2 individuals went zero
Iterations:   7%|▋         | 2/30 [02:22<33:04, 70.87s/iteration] [ 2023-08-21 11:28:26,805 ] 1 individuals went zero
 [ 2023-08-21 11:28:52,640 ] 1 individuals went zero
Iterations:  10%|█         | 3/30 [03:31<31:35, 70.19s/iteration] [ 2023-08-21 11:29:17,312 ] 1 individuals went zero
 [ 2023-08-21 11:29:22,635 ] 1 individuals went zero
 [ 2023-08-21 11:29:26,358 ] 1 individuals went zero
 [ 2023-08-21 11:29:52,146 ] 1 individuals went zero
 [ 2023-08-21 11:29:55,970 ] 1 individuals went zero
 [ 2023-08-21 11:29:57,145 ] 1 individuals went zero
 [ 2023-08-21 11:29:59,969 ] 1 individuals went zero
 [ 2023-08-21 11:30:00,697 ] 1 individuals went zero
Iterations:  13%|█▎        | 4/30 [04:40<30:11, 69.66s/iteration] [ 2023-08-21 11:30:48,193 ] 1 individuals went zer

In [24]:
da_features

[['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age', 'hypertension', 'ever_married'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age'],
 ['age']]

In [25]:
Stroke_data =pd.DataFrame(X['age'])
Stroke_data['stroke']=y

In [26]:
predictor_combined = TabularPredictor(
    label="stroke", 
    
).fit(
    train_data=Stroke_data, 

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_120536\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_120536\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.89 GB / 255.46 GB (12.9%)
Train Data Rows:    9722
Train Data Columns: 1
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1791.22 MB


In [27]:
Stroke_data =pd.DataFrame(X[['age', 'hypertension', 'ever_married']])
Stroke_data['stroke']=y

In [28]:
predictor_combined = TabularPredictor(
    label="stroke", 
    
).fit(
    train_data=Stroke_data, 

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_120837\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_120837\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.84 GB / 255.46 GB (12.9%)
Train Data Rows:    9722
Train Data Columns: 3
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1551.36 MB


## DataSet 2

In [30]:
stroke2 = pd.read_csv('stroke/stroke2.csv')

In [31]:
stroke2=preprocess_dataset(stroke2)
X = stroke2.drop(['stroke'],axis=1)
y = stroke2['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [32]:
num_iterations = 30 
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations:   7%|▋         | 2/30 [01:28<20:42, 44.39s/iteration] [ 2023-08-21 12:16:47,227 ] 1 individuals went zero
 [ 2023-08-21 12:16:51,298 ] 1 individuals went zero
Iterations:  23%|██▎       | 7/30 [05:25<18:03, 47.12s/iteration] [ 2023-08-21 12:21:07,383 ] 1 individuals went zero
 [ 2023-08-21 12:21:16,765 ] 1 individuals went zero
Iterations:  90%|█████████ | 27/30 [21:04<02:18, 46.25s/iteration] [ 2023-08-21 12:36:32,818 ] 1 individuals went zero
 [ 2023-08-21 12:36:36,832 ] 1 individuals went zero
 [ 2023-08-21 12:36:47,356 ] 1 individuals went zero
 [ 2023-08-21 12:36:49,830 ] 1 individuals went zero
Iterations: 100%|██████████| 30/30 [23:26<00:00, 46.90s/iteration]


In [33]:
features

[['gender', 'Residence_type'],
 ['age', 'heart_disease', 'ever_married'],
 ['age', 'ever_married', 'work_type'],
 ['gender', 'ever_married', 'work_type', 'Residence_type'],
 ['age', 'work_type'],
 ['gender', 'heart_disease'],
 ['gender'],
 ['hypertension', 'heart_disease', 'ever_married'],
 ['heart_disease', 'avg_glucose_level'],
 ['gender', 'hypertension', 'ever_married'],
 ['gender', 'ever_married'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['gender', 'ever_married'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['age', 'work_type'],
 ['gender', 'ever_married', 'work_type', 'Residence_type'],
 ['age', 'avg_glucose_level', 'bmi'],
 ['gender', 'ever_married', 'work_type', 'Residence_type'],
 ['age', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'Residence_type'],
 ['gender', 'heart_disease', 'Residence_type'

In [34]:
Stroke_data =pd.DataFrame(X[['gender', 'age', 'ever_married', 'avg_glucose_level', 'bmi']])
Stroke_data['stroke']=y

In [35]:
predictor_combined = TabularPredictor(
    label="stroke", 
    
).fit(
    train_data=Stroke_data, 

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_124028\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_124028\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.87 GB / 255.46 GB (12.9%)
Train Data Rows:    9466
Train Data Columns: 5
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1650.27 MB


[1000]	valid_set's binary_error: 0.123548
[2000]	valid_set's binary_error: 0.101373
[3000]	valid_set's binary_error: 0.0918691
[4000]	valid_set's binary_error: 0.0887012


	0.9124	 = Validation score   (accuracy)
	5.37s	 = Training   runtime
	0.26s	 = Validation runtime
Fitting model: LightGBM ...
	0.9493	 = Validation score   (accuracy)
	1.06s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9219	 = Validation score   (accuracy)
	1.36s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9145	 = Validation score   (accuracy)
	2.25s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.9398	 = Validation score   (accuracy)
	6.89s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9134	 = Validation score   (accuracy)
	0.94s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9092	 = Validation score   (accuracy)
	1.04s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8332	 = Validation score   (accuracy)
	11.88s	 = Training   runtime
	0.02s	

In [22]:
from sklearn.neighbors import KNeighborsClassifier

def run_da_optimization(num_iterations, X_train, y_train, X_test, y_test):
    da_results_per_iteration = []
    da_best_feature_lists = []
    da_best_dims = []
    da_best_scores = []
    
    for _ in tqdm(range(num_iterations), desc="Iterations", unit="iteration"):
        # Replace with your actual algorithm import and instantiation
        algo_object = DragonFlyOptimization(objective_function_topass,
                                            n_iteration=100,
                                            population_size=11,
                                            minimize=True,
                                            logger=None)
        
        model = KNeighborsClassifier()
        
        algo_object.fit(model, X_train, y_train, X_test, y_test, verbose=False)
    
        da_iteration_results = {
            "results_per_iteration": algo_object.best_results_per_iteration,
            "best_dim_list": algo_object.best_dim.tolist(),
            "best_feature_list": algo_object.best_feature_list,
            "best_score": algo_object.best_score
        }
        da_best_feature_list = algo_object.best_feature_list
        da_best_dim = algo_object.best_dim
        da_best_score = algo_object.best_score

        da_results_per_iteration.append(da_iteration_results)
        da_best_feature_lists.append(da_best_feature_list)
        da_best_dims.append(da_best_dim.tolist())
        da_best_scores.append(da_best_score)
    
    return da_results_per_iteration, da_best_feature_lists, da_best_dims, da_best_scores



In [36]:
num_iterations = 30
da_results, da_features, da_dims, da_scores = run_da_optimization(num_iterations, X_train, y_train, X_test, y_test)


Iterations:   0%|          | 0/30 [00:00<?, ?iteration/s] [ 2023-08-21 12:44:02,413 ] 1 individuals went zero
 [ 2023-08-21 12:44:10,368 ] 1 individuals went zero
 [ 2023-08-21 12:44:22,739 ] 1 individuals went zero
Iterations:   7%|▋         | 2/30 [02:14<31:31, 67.57s/iteration] [ 2023-08-21 12:46:28,113 ] 1 individuals went zero
 [ 2023-08-21 12:46:59,709 ] 1 individuals went zero
 [ 2023-08-21 12:47:00,210 ] 1 individuals went zero
 [ 2023-08-21 12:47:05,249 ] 1 individuals went zero
Iterations:  10%|█         | 3/30 [03:25<31:02, 68.99s/iteration] [ 2023-08-21 12:47:10,926 ] 1 individuals went zero
 [ 2023-08-21 12:47:14,544 ] 1 individuals went zero
Iterations:  13%|█▎        | 4/30 [04:33<29:46, 68.69s/iteration] [ 2023-08-21 12:48:15,510 ] 1 individuals went zero
 [ 2023-08-21 12:49:13,047 ] 1 individuals went zero
 [ 2023-08-21 12:49:16,502 ] 1 individuals went zero
 [ 2023-08-21 12:49:17,275 ] 1 individuals went zero
Iterations:  17%|█▋        | 5/30 [05:39<28:12, 67.70s/iter

In [37]:
da_features

[['age', 'work_type'],
 ['age', 'work_type'],
 ['age'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type'],
 ['age', 'work_type']]

In [38]:
Stroke_data =pd.DataFrame(X[['age', 'work_type']])
Stroke_data['stroke']=y

In [39]:
predictor_combined = TabularPredictor(
    label="stroke", 
    
).fit(
    train_data=Stroke_data, 

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230821_132030\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230821_132030\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   32.61 GB / 255.46 GB (12.8%)
Train Data Rows:    9466
Train Data Columns: 2
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1610.53 MB


## Dataset 3

In [40]:
stroke3 = pd.read_csv('stroke/stroke3.csv')

In [41]:
stroke3=preprocess_dataset(stroke2)
X = stroke3.drop(['stroke'],axis=1)
y = stroke3['stroke']

from imblearn.over_sampling import SMOTE
# Assuming X and y are your feature matrix and target variable, respectively
smote = SMOTE()
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [45]:
num_iterations = 30 
results, features, dims, scores = run_cgwoda_optimization(num_iterations, X_train, y_train, X_test, y_test)

Iterations:  20%|██        | 6/30 [54:06<3:38:31, 546.33s/iteration] [ 2023-08-21 14:23:29,664 ] 1 individuals went zero
 [ 2023-08-21 14:28:16,242 ] 1 individuals went zero
 [ 2023-08-21 14:28:55,177 ] 1 individuals went zero
Iterations:  47%|████▋     | 14/30 [2:07:30<2:27:40, 553.77s/iteration] [ 2023-08-21 15:38:59,989 ] 1 individuals went zero
 [ 2023-08-21 15:42:18,262 ] 1 individuals went zero
Iterations:  73%|███████▎  | 22/30 [3:25:40<1:16:12, 571.62s/iteration] [ 2023-08-21 16:58:10,011 ] 1 individuals went zero
 [ 2023-08-21 17:00:31,035 ] 1 individuals went zero
 [ 2023-08-21 17:01:51,730 ] 1 individuals went zero
 [ 2023-08-21 17:02:30,574 ] 1 individuals went zero
 [ 2023-08-21 17:02:59,529 ] 2 individuals went zero
 [ 2023-08-21 17:03:05,459 ] 1 individuals went zero
Iterations: 100%|██████████| 30/30 [19:53:40<00:00, 2387.34s/iteration]    


In [46]:
features

[['heart_disease', 'Residence_type', 'bmi'],
 ['gender', 'hypertension', 'bmi'],
 ['hypertension', 'heart_disease', 'ever_married', 'smoking_status'],
 ['heart_disease'],
 ['ever_married', 'bmi'],
 ['Residence_type', 'bmi'],
 ['Residence_type', 'bmi'],
 ['hypertension', 'heart_disease', 'ever_married', 'smoking_status'],
 ['hypertension', 'ever_married', 'work_type'],
 ['Residence_type', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['hypertension', 'bmi'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['Residence_type', 'bmi'],
 ['work_type'],
 ['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status'],
 ['ever_married', 'work_type'],
 ['gender'],
 ['hypertension', 'bmi'],
 ['hypertension', 'bmi'],
 ['hypertension', 'ever_married'],
 ['heart_disease', 'Residence_type'],
 ['bmi'],
 ['heart_disease', 'ever_married', 'bmi'],
 ['gende

In [47]:
Stroke_data =pd.DataFrame(X[['age',
  'ever_married',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status']])
Stroke_data['stroke']=y

In [48]:
predictor_combined = TabularPredictor(
    label="stroke", 
    
).fit(
    train_data=Stroke_data, 

)

No path specified. Models will be saved in: "AutogluonModels\ag-20230822_092619\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230822_092619\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   31.29 GB / 255.46 GB (12.2%)
Train Data Rows:    85234
Train Data Columns: 6
Label Column: stroke
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1548.78 MB

[1000]	valid_set's binary_error: 0.1088
[2000]	valid_set's binary_error: 0.0784
[3000]	valid_set's binary_error: 0.068
[4000]	valid_set's binary_error: 0.0568
[5000]	valid_set's binary_error: 0.0524
[6000]	valid_set's binary_error: 0.0484
[7000]	valid_set's binary_error: 0.0468
[8000]	valid_set's binary_error: 0.046
[9000]	valid_set's binary_error: 0.0432
[10000]	valid_set's binary_error: 0.0408


	0.9592	 = Validation score   (accuracy)
	50.87s	 = Training   runtime
	2.36s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0256
[2000]	valid_set's binary_error: 0.0216


	0.9784	 = Validation score   (accuracy)
	11.48s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9756	 = Validation score   (accuracy)
	17.27s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9748	 = Validation score   (accuracy)
	29.85s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: CatBoost ...
	0.9796	 = Validation score   (accuracy)
	110.91s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.972	 = Validation score   (accuracy)
	3.6s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9692	 = Validation score   (accuracy)
	3.6s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8596	 = Validation score   (accuracy)
	145.33s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: XGBoost ...
	0.9684	 = Validation score   (accuracy)
	10.6s	 = Training   runtime
	0.1

In [50]:
predictor_combined.leaderboard()

                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.9844       3.022124  209.405931                0.004718           1.470291            2       True         14
1              CatBoost     0.9796       0.018949  110.913519                0.018949         110.913519            1       True          7
2              LightGBM     0.9784       0.183970   11.477721                0.183970          11.477721            1       True          4
3         LightGBMLarge     0.9764       0.138153    6.802608                0.138153           6.802608            1       True         13
4      RandomForestGini     0.9756       0.184586   17.273694                0.184586          17.273694            1       True          5
5      RandomForestEntr     0.9748       0.183279   29.845453                0.183279          29.845453            1       True          6
6        ExtraTreesG

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9844,3.022124,209.405931,0.004718,1.470291,2,True,14
1,CatBoost,0.9796,0.018949,110.913519,0.018949,110.913519,1,True,7
2,LightGBM,0.9784,0.183970,11.477721,0.183970,11.477721,1,True,4
3,LightGBMLarge,0.9764,0.138153,6.802608,0.138153,6.802608,1,True,13
4,RandomForestGini,0.9756,0.184586,17.273694,0.184586,17.273694,1,True,5
5,RandomForestEntr,0.9748,0.183279,29.845453,0.183279,29.845453,1,True,6
6,ExtraTreesGini,0.9720,0.103960,3.604559,0.103960,3.604559,1,True,8
7,ExtraTreesEntr,0.9692,0.075672,3.604416,0.075672,3.604416,1,True,9
8,XGBoost,0.9684,0.129939,10.595365,0.129939,10.595365,1,True,11
9,LightGBMXT,0.9592,2.361809,50.872734,2.361809,50.872734,1,True,3
